### Encadeamento Simples

In [1]:
#!pip install langchain

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# Permite usar Runnables no Lugar de Funções
from langchain_core.runnables import RunnableLambda

# A RunnableSequence constructed using the `|` operator -> o operador `|` é o operador de composição de funções, o resultado da primeira é passada para a segunda
sequence: RunnableLambda = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
# Invocando a sequência 
sequence.invoke(1) # 4
# Roda "invoke" em paralelo usando uma thread poll -> Subclasses devem dar override se conseguirem ser mais eficientes
sequence.batch([1, 2, 3]) # [4, 6, 8]

# A sequence that contains a RunnableParallel constructed using a dict literal
# 1 - Lambda | dict {lambda1, lambda2} -> 2 saídas
sequence: RunnableLambda = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}

# Descreve uma relação em que uma entrada é transformada em duas saídas
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}

{'mul_2': 4, 'mul_5': 10}

## Outro Exemplo de Chain

In [5]:
from langchain_core.runnables import chain

@chain
def step1(data: str) -> str:
    return data.upper()

@chain
def step2(data: str) -> str:
    return f"Processed: {data}"

# Cria uma cadeia de execução, saída de step1 é passada para step2
runnable_chain: chain = step1 | step2  # Composição de Steps

output = runnable_chain.invoke("hello")  # Executa em cadeia
print(output)  # Output será "Processed: HELLO"

Processed: HELLO


In [ ]:
from langchain_core.runnables import RunnableLambda

import random

def add_one(x: int) -> int:
    return x + 1

def buggy_double(y: int) -> int:
    '''Buggy code that will fail 70% of the time'''
    if random.random() > 0.3:
        print('This code failed, and will probably be retried!')  # noqa: T201
        raise ValueError('Triggered buggy code')
    return y * 2

sequence = (
    RunnableLambda(add_one) |
    RunnableLambda(buggy_double).with_retry( # Retry on failure
        stop_after_attempt=10,
        wait_exponential_jitter=False
    )
)

print(sequence.input_schema.schema()) # Show inferred input schema
print(sequence.output_schema.schema()) # Show inferred output schema
print(sequence.invoke(2)) # invoke the sequence (note the retry above!!)